In [1]:
from claude_agent_graph import AgentGraph

# Create a graph
graph = AgentGraph(name="my_network")

# Add nodes (agents)
await graph.add_node(
    node_id="supervisor",
    system_prompt="You coordinate worker agents.",
    model="claude-sonnet-4-20250514"
)

await graph.add_node(
    node_id="worker_1",
    system_prompt="You execute tasks.",
    model="claude-sonnet-4-20250514"
)

# Add edge (connection)
await graph.add_edge(
    from_node="supervisor",
    to_node="worker_1",
    directed=True
)

TypeError: object Node can't be used in 'await' expression

In [ ]:

# Send message
# await graph.send_message(
#     from_node="supervisor",
#     to_node="worker_1",
#     content="Analyze the data"
# )